# SnpArray

`SnpArray` is an array of `Tuple{Bool,Bool}` and adopts the same coding as the [Plink binary format](http://zzz.bwh.harvard.edu/plink/binary.shtml). If `A1` and `A2` are the two alleles, the coding rule is  

| Genotype | SnpArray |  
|:---:|:---:|  
| A1,A1 | (false,false) |  
| A1,A2 | (false,true) |  
| A2,A2 | (true,true) |  
| missing | (true,false) |  

The code `(true,false)` is reserved for missing genotype. Otherwise, the bit `true` represents one copy of allele `A2`. In a two-dimensional `SnpArray`, each row is a person and each column is a SNP.

For complete genotype data, for example, after imputation, consider using the [HaplotypeArray](@ref) type.

## Constructor

There are various ways to initialize a `SnpArray`.  

* `SnpArray` can be initialized from [Plink binary files](http://zzz.bwh.harvard.edu/plink/binary.shtml), say the sample data set hapmap3:

In [49]:
;ls -al "hapmap3.*"

-rw-r--r--  1 huazhou  staff  1128171 Jun 19 14:43 hapmap3.bed
-rw-r--r--  1 huazhou  staff   388672 Jun 19 14:43 hapmap3.bim
-rw-r--r--  1 huazhou  staff     7136 Jun 19 14:43 hapmap3.fam
-rw-r--r--  1 huazhou  staff   332960 Jun 19 14:43 hapmap3.map


In [50]:
using SnpArrays
hapmap = SnpArray("hapmap3")

324×13928 SnpArrays.SnpArray{2}:
 (true, true)  (true, true)   (false, false)  …  (true, true)   (true, true)
 (true, true)  (false, true)  (false, true)      (false, true)  (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (false, true)  (true, true)    …  (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (false, false)     (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)   …  (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (false, false

By default, the constructor figures out the number of individuals and SNPs from the `.bim` and `.fam` files.

In [51]:
# rows are people; columns are SNPs
people, snps = size(hapmap)

(324, 13928)

Alternatively, users can supply keyword arguments `people` and `snps` to the constructor. In this case only the `.bed` file needs to be present.

In [52]:
hapmap = SnpArray("hapmap3"; people = 324, snps = 13928)

324×13928 SnpArrays.SnpArray{2}:
 (true, true)  (true, true)   (false, false)  …  (true, true)   (true, true)
 (true, true)  (false, true)  (false, true)      (false, true)  (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (false, true)  (true, true)    …  (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (false, false)     (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)   …  (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (false, false

Internally `SnpArray` stores data as `BitArray`s and consumes approximately the same amount of memory as the Plink `bed` file size.

In [53]:
# memory usage, bed file size
Base.summarysize(hapmap), filesize("hapmap3.bed")

(1128256, 1128171)

* `SnpArray` can be initialized from a matrix of A1 allele counts.

In [54]:
SnpArray(rand(0:2, 5, 3))

5×3 SnpArrays.SnpArray{2}:
 (false, false)  (false, false)  (false, true) 
 (true, true)    (true, true)    (false, true) 
 (false, true)   (false, false)  (false, false)
 (false, false)  (false, true)   (false, true) 
 (true, true)    (false, false)  (false, true) 

* `SnpArray(m, n)` generates an m by n `SnpArray` of all A1 alleles.

In [55]:
s = SnpArray(5, 3)

5×3 SnpArrays.SnpArray{2}:
 (false, false)  (false, false)  (false, false)
 (false, false)  (false, false)  (false, false)
 (false, false)  (false, false)  (false, false)
 (false, false)  (false, false)  (false, false)
 (false, false)  (false, false)  (false, false)

## Summary statistics

`summarize` function computes the following summary statistics of a `SnpArray`:  

* `maf`: minor allele frequencies, taking into account of missingness.  
* `minor_allele`: a `BitVector` indicating the minor allele for each SNP.   `minor_allele[j]==true` means A1 is the minor allele for SNP j; `minor_allele[j]==false` means A2 is the minor allele for SNP j.  
* `missings_by_snp`: number of missing genotypes for each snp.  
* `missings_by_person`: number of missing genotypes for each person.  

In [56]:
maf, minor_allele, missings_by_snp, missings_by_person = summarize(hapmap)
# minor allele frequencies
maf'

1×13928 RowVector{Float64,Array{Float64,1}}:
 0.0  0.0776398  0.324074  0.191589  …  0.00154321  0.0417957  0.00617284

In [57]:
# total number of missing genotypes
sum(missings_by_snp), sum(missings_by_person)

(11894, 11894)

In [58]:
# proportion of missing genotypes
sum(missings_by_snp) / length(hapmap)

0.0026356890108565393

## Filtering

In almost all analyses, SNPs and individuals with low genotyping success rates are ignored. This filtering step is an important tool for removing likely false positives from association testing, as genotyping failure often occurs preferentially in cases or controls, or is correlated with the quantitative trait. `filter(s, min_success_rate_per_snp, min_success_rate_per_person)` does filtering according to the specified success rates for SNPs and people. Default is 0.98 for both.

In [59]:
# filtering SNPs and people to have both success rates above 0.98
snp_idx, person_idx = filter(hapmap, 0.98, 0.98)
# summary statistics of the filtered SnpArray
_, _, missings_by_snp_filtered, missings_by_person_filtered = summarize(hapmap[person_idx, snp_idx]);

In [60]:
# minimum SNP genotyping success rate after filtering ≥ 0.98
1.0 - maximum(missings_by_snp_filtered) / length(missings_by_person_filtered)

0.9813084112149533

In [61]:
# minimum person genotyping success rate after filtering ≥ 0.98
1.0 - maximum(missings_by_person_filtered) / length(missings_by_snp_filtered)

0.9818511796733213

## Random genotypes generation

`randgeno(a1freq)` generates a random genotype according to A1 allele frequency `a1freq`.

In [62]:
randgeno(0.5)

(false, true)

`randgeno(maf, minor_allele)` generates a random genotype according to minor allele frequency `maf` and whether the minor allele is A1 (`minor_allele==true`) or A2 (`minor_allele==false`).

In [63]:
randgeno(0.25, true)

(true, true)

`randgeno(n, maf, minor_allele)` generates a vector of random genotypes according to a common minor allele frequency `maf` and the minor allele.

In [64]:
randgeno(10, 0.25, true)

10-element SnpArrays.SnpArray{1}:
 (false, true)
 (true, true) 
 (true, true) 
 (false, true)
 (false, true)
 (false, true)
 (false, true)
 (true, true) 
 (false, true)
 (true, true) 

`randgeno(m, n, maf, minor_allele)` generates a random $m$-by-$n$ `SnpArray` according to a vector of minor allele frequencies `maf` and a minor allele indicator vector. The lengths of both vectors should be `n`.

In [65]:
# this is a random replicate of the hapmap data
randgeno(size(hapmap), maf, minor_allele)

324×13928 SnpArrays.SnpArray{2}:
 (true, true)  (true, true)   (true, true)    …  (true, true)   (true, true)
 (true, true)  (false, true)  (true, true)       (false, true)  (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (false, true)  (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)    …  (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (false, true)  (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)   …  (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (false, true)  (true, true) 

## Subsetting

Subsetting a `SnpArray` works the same way as subsetting any other arrays.

In [66]:
# genotypes of the 1st person
hapmap[1, :]

13928-element SnpArrays.SnpArray{1}:
 (true, true)  
 (true, true)  
 (false, false)
 (true, true)  
 (true, true)  
 (true, true)  
 (false, true) 
 (false, true) 
 (true, true)  
 (false, true) 
 (true, true)  
 (true, true)  
 (false, false)
 ⋮             
 (false, true) 
 (false, true) 
 (true, true)  
 (false, true) 
 (false, true) 
 (false, true) 
 (false, true) 
 (false, true) 
 (false, true) 
 (true, true)  
 (true, true)  
 (true, true)  

In [67]:
# genotypes of the 5th SNP
hapmap[:, 5]

324-element SnpArrays.SnpArray{1}:
 (true, true)  
 (true, true)  
 (false, true) 
 (false, true) 
 (true, true)  
 (false, false)
 (false, false)
 (true, true)  
 (true, true)  
 (true, true)  
 (true, true)  
 (true, true)  
 (false, true) 
 ⋮             
 (false, false)
 (true, true)  
 (false, true) 
 (true, true)  
 (true, true)  
 (true, true)  
 (true, true)  
 (true, true)  
 (false, true) 
 (true, true)  
 (true, true)  
 (true, true)  

In [68]:
# subsetting both persons and SNPs
hapmap[1:5, 5:10]

5×6 SnpArrays.SnpArray{2}:
 (true, true)   (true, true)  (false, true)  …  (true, true)   (false, true)
 (true, true)   (true, true)  (true, true)      (true, true)   (false, true)
 (false, true)  (true, true)  (true, true)      (false, true)  (true, true) 
 (false, true)  (true, true)  (true, true)      (true, true)   (false, true)
 (true, true)   (true, true)  (true, true)      (true, true)   (false, true)

In [69]:
# filter out rare SNPs with MAF < 0.05
hapmap[:, maf .≥ 0.05]

324×12085 SnpArrays.SnpArray{2}:
 (true, true)   (false, false)  …  (false, true)  (false, true)
 (false, true)  (false, true)      (true, true)   (true, true) 
 (true, true)   (false, true)      (true, true)   (true, true) 
 (true, true)   (false, true)      (false, true)  (false, true)
 (true, true)   (false, true)      (true, true)   (true, true) 
 (false, true)  (true, true)    …  (false, true)  (false, true)
 (true, true)   (true, true)       (true, true)   (true, true) 
 (true, true)   (false, false)     (true, true)   (true, true) 
 (true, true)   (false, true)      (true, true)   (true, true) 
 (true, true)   (false, true)      (false, true)  (false, true)
 (true, true)   (false, true)   …  (true, true)   (true, true) 
 (true, true)   (true, true)       (false, true)  (false, true)
 (true, true)   (false, false)     (false, true)  (false, true)
 ⋮                              ⋱                              
 (true, true)   (false, true)      (false, true)  (false, true)
 (true,

In [70]:
# filter out individuals with genotyping success rate < 0.90
hapmap[missings_by_person / people .< 0.1, :]

220×13928 SnpArrays.SnpArray{2}:
 (true, true)  (true, true)   (false, false)  …  (true, true)   (true, true)
 (true, true)  (false, true)  (false, true)      (false, true)  (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (false, false)     (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)   …  (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (true, true)   (false, false)     (true, true)   (true, true)
 (true, true)  (true, true)   (false, true)      (true, true)   (true, true)
 (true, true)  (false, true)  (false, true)   …  (true, true)   (true, true)
 (true, true)  (true, true)   (true, true)       (true, true)   (true, true)
 (true, true)  (true, true)   (true, true) 

`view` creates views of subarray without copying data and improve efficiency in many calculations.

In [71]:
mafcommon, = summarize(view(hapmap, :, maf .≥ 0.05))
mafcommon'

1×12085 RowVector{Float64,Array{Float64,1}}:
 0.0776398  0.324074  0.191589  …  0.310937  0.23913  0.23913  0.23913

## Assignment

It is possible to assign specific genotypes to a `SnpArray` entry.

In [72]:
hapmap[1, 1]

(true, true)

In [73]:
hapmap[1, 1] = (false, true)
hapmap[1, 1]

(false, true)

In [74]:
hapmap[1, 1] = NaN
hapmap[1, 1]

(true, false)

In [75]:
hapmap[1, 1] = 2
hapmap[1, 1]

(true, true)

Subsetted assignment such as `hapmap[:, 1] = Nan` is also valid.

## Convert, copy and imputation

In most analyses we convert a whole `SnpArray` or slices of it to numeric arrays (matrix of **minor allele counts**) for statistical analysis. Keep in mind that the storage of resultant data can be up to 32 fold larger than that of the original `SnpArray`. Fortunately, rich collection of data types in `Julia` allow us choose one that fits into memory. Below are estimates of memory usage for some common data types with `n` persons and `p` SNPs. Here MAF denotes the **average** minor allele frequencies.

* `SnpArray`: $0.25np$ bytes  
* `Matrix{Int8}`: $np$ bytes  
* `Matrix{Float16}`: $2np$ bytes  
* `Matrix{Float32}`: $4np$ bytes  
* `Matrix{Float64}`: $8np$ bytes  
* `SparseMatrixCSC{Float64,Int64}`: $16 \cdot \text{NNZ} + 8(p+1) \approx 16np(2\text{MAF}(1-\text{MAF})+\text{MAF}^2) + 8(p+1) = 16np \cdot \text{MAF}(2-\text{MAF}) + 8(p+1)$ bytes. When the average MAF=0.25, this is about $7np$ bytes. When MAF=0.025, this is about $0.8np$ bypes, 10 fold smaller than the `Matrix{Float64}` type.  
* `SparseMatrixCSC{Int8,UInt32}`: $5 \cdot \text{NNZ} + 4(p+1) \approx 5np(2\text{MAF}(1-\text{MAF})+\text{MAF}^2) + 4(p+1) = 5np \cdot \text{MAF}(2-\text{MAF}) + 4(p+1)$ bytes. When the average MAF=0.25, this is about $2.2np$ bytes. When MAF=0.08, this is about $0.8np$ bypes, 10 fold smaller than `Matrix{Float64}` type.  
* Two `SparseMatrixCSC{Bool,Int64}`: $2np \cdot \text{MAF} \cdot 9 + 16(p+1) = 18 np \cdot \text{MAF} + 16(p+1)$ bytes. When the average MAF=0.25, this is about $4.5np$ bytes. When MAF=0.045, this is about $0.8np$ bytes, 10 fold smaller than `Matrix{Float64}` type.  

To be concrete, consider 2 typical data sets:  
* COPD (GWAS): $n = 6670$ individuals, $p = 630998$ SNPs, average MAF is 0.2454.
* GAW19 (sequencing study): $n = 959$ individuals, $p = 8348674$ SNPs, average MAF is 0.085.  

| Data Type | COPD | GAW19 |  
|---|---:|---:|  
| `SnpArray` | 1.05GB | 2GB |  
| `Matrix{Float64}` | 33.67GB | 64.05GB |  
| `SparseMatrixCSC{Float64,Int64}` | 29GB | 20.82GB |  
| `SparseMatrixCSC{Bool,Int64}` | 18.6GB | 12.386GB |  

Apparently for data sets with a majority of rare variants, converting to sparse matrices saves memory and often brings computational advantages too. In the `SparseMatrixCSC` format, the integer type of the row indices `rowval` and column pointer `colptr` should have maximal allowable value larger than the number of nonzeros in the matrix. The `InexactError()` error encountered during conversion often indicates that the integer type has a too small range. The utility function `estimatesize` conveniently estimates memory usage in bytes for the input data type.

In [76]:
# estimated memory usage if convert to Matrix{Float64}
estimatesize(people, snps, Matrix{Float64})

3.6101376e7

In [77]:
# convert to Matrix{Float64}
hapmapf64 = convert(Matrix{Float64}, hapmap)

324×13928 Array{Float64,2}:
 0.0  0.0  2.0  0.0  0.0  0.0  1.0  1.0  …  1.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0  1.0  0.0  0.0  2.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0     1.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  2.0  0.0  0.0  0.0  …  2.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  2.0  0.0  0.0  2.0     1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  2.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0     1.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  2.0     1.0  1.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  2.0  0.0  1.0  0.0  0.0  

In [78]:
# actual memory usage of Matrix{Float64}
Base.summarysize(hapmapf64)

36101376

In [79]:
# average maf of the hapmap3 data set
mean(maf)

0.222585591341583

In [80]:
# estimated memory usage if convert to SparseMatrixCSC{Float32, UInt32} matrix
estimatesize(people, snps, SparseMatrixCSC{Float32, UInt32}, mean(maf))

1.4338389205819245e7

In [81]:
# convert to SparseMatrixCSC{Float32, UInt32} matrix
hapmapf32sp = convert(SparseMatrixCSC{Float32, UInt32}, hapmap)

324×13928 SparseMatrixCSC{Float32,UInt32} with 1614876 stored entries:
  [2    ,     2]  =  1.0
  [6    ,     2]  =  1.0
  [15   ,     2]  =  1.0
  [31   ,     2]  =  1.0
  [33   ,     2]  =  1.0
  [35   ,     2]  =  1.0
  [43   ,     2]  =  1.0
  [44   ,     2]  =  1.0
  [50   ,     2]  =  1.0
  [54   ,     2]  =  1.0
  ⋮
  [135  , 13927]  =  1.0
  [148  , 13927]  =  1.0
  [160  , 13927]  =  1.0
  [164  , 13927]  =  2.0
  [167  , 13927]  =  1.0
  [185  , 13927]  =  1.0
  [266  , 13927]  =  1.0
  [280  , 13927]  =  1.0
  [288  , 13927]  =  1.0
  [118  , 13928]  =  2.0
  [231  , 13928]  =  2.0

In [82]:
# actual memory usage if convert to SparseMatrixCSC{Float32, UInt32} matrix
Base.summarysize(hapmapf32sp)

12974764

By default the `convert()` method converts missing genotypes to `NaN`.

In [83]:
# number of missing genotypes
countnz(isnan.(hapmap)), countnz(isnan.(hapmapf64))

(11894, 11894)

One can enforce **crude imputation** by setting the optional argument `impute=true`. Imputation is done by generating two random alleles according to the minor allele frequency. This is a neutral but not an optimal strategy, and users should impute missing genotypes by more advanced methods.

In [84]:
hapmapf64impute = convert(Matrix{Float64}, hapmap; impute = true)
countnz(isnan.(hapmapf64impute))

0

By default `convert()` translates genotypes according to the *additive* SNP model, which essentially counts the number of **minor allele** (0, 1 or 2) per genotype. Other SNP models are *dominant* and *recessive*, both in terms of the **minor allele**. When `A1` is the minor allele, genotypes are translated to real number according to

| Genotype | `SnpArray` | `model=:additive` | `model=:dominant` | `model=:recessive` |    
|:---:|:---:|:---:|:---:|:---:|  
| A1,A1 | (false,false) | 2 | 1 | 1 |  
| A1,A2 | (false,true) | 1 | 1 | 0 |  
| A2,A2 | (true,true) | 0 | 0 | 0 |  
| missing | (true,false) | NaN | NaN | NaN | 

When `A2` is the minor allele, genotypes are translated according to

| Genotype | `SnpArray` | `model=:additive` | `model=:dominant` | `model=:recessive` |    
|:---:|:---:|:---:|:---:|:---:|  
| A1,A1 | (false,false) | 0 | 0 | 0 |  
| A1,A2 | (false,true) | 1 | 1 | 0 |  
| A2,A2 | (true,true) | 2 | 1 | 1 |  
| missing | (true,false) | NaN | NaN | NaN |

In [85]:
[convert(Vector{Float64}, hapmap[1:10, 5]; model = :additive) convert(Vector{Float64}, hapmap[1:10, 5]; model = :dominant) convert(Vector{Float64}, hapmap[1:10, 5]; model = :recessive)]

10×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 1.0  1.0  0.0
 1.0  1.0  0.0
 0.0  0.0  0.0
 2.0  1.0  1.0
 2.0  1.0  1.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

By default `convert()` does **not** center and scale genotypes. Setting the optional arguments `center=true, scale=true` centers genotypes at 2MAF and scales them by $[2 \cdot \text{MAF} \cdot (1 - \text{MAF})]^{-1/2}$. Mono-allelic SNPs (MAF=0) are not scaled.

In [86]:
[convert(Vector{Float64}, hapmap[:, 5]) convert(Vector{Float64}, hapmap[:, 5]; center = true, scale = true)]

324×2 Array{Float64,2}:
 0.0  -1.25702 
 0.0  -1.25702 
 1.0   0.167017
 1.0   0.167017
 0.0  -1.25702 
 2.0   1.59106 
 2.0   1.59106 
 0.0  -1.25702 
 0.0  -1.25702 
 0.0  -1.25702 
 0.0  -1.25702 
 0.0  -1.25702 
 1.0   0.167017
 ⋮             
 2.0   1.59106 
 0.0  -1.25702 
 1.0   0.167017
 0.0  -1.25702 
 0.0  -1.25702 
 0.0  -1.25702 
 0.0  -1.25702 
 0.0  -1.25702 
 1.0   0.167017
 0.0  -1.25702 
 0.0  -1.25702 
 0.0  -1.25702 

`copy!()` is the in-place version of `convert()`. Options such as GWAS loop over SNPs and perform statistical anlaysis for each SNP. This can be achieved by

In [87]:
g = zeros(people)
for j = 1:snps
    copy!(g, hapmap[:, j]; model = :additive, impute = true)
    # do statistical anlaysis
end

## Empirical kinship matrix

`grm` function computes the empirical kinship matrix using either the genetic relationship matrix, `grm(A, model=:GRM)`, or the method of moment method, `grm(A, model=:MoM)`. 

!!! note

    Missing genotypes are imputed according to minor allele frequencies on the fly.  
    


!!! note

    It is often necessary to filter SNPs according to minor allele frequency and LD before calculating empirical kinship matrix.  


By default, `grm` exlcude SNPs with minor allele frequency below 0.01. This can be changed by the keyword argument `maf_threshold`.

In [88]:
# GRM using all SNPs with MAF ≥ 0.01. Same as
# grm(hapmap; maf_threshold = 0.01)
grm(hapmap)

324×324 Array{Float64,2}:
 0.571462   0.0465067  0.0198863  …  0.065456   0.0716354  0.0653478
 0.0465067  0.544896   0.028444      0.0515233  0.0456809  0.0633915
 0.0198863  0.028444   0.520507      0.046051   0.0302884  0.0366246
 0.0484459  0.0369766  0.0293501     0.060103   0.0660171  0.0601609
 0.052807   0.0424252  0.0264585     0.0718885  0.0583893  0.0658839
 0.0453662  0.0316632  0.0382386  …  0.070662   0.0566399  0.0650759
 0.0401015  0.022318   0.0119087     0.044437   0.0381739  0.0372612
 0.0410669  0.0386841  0.0210986     0.0574887  0.0547491  0.0663823
 0.0300075  0.0309477  0.0172097     0.0340103  0.0460311  0.0382384
 0.0394993  0.0422916  0.0261469     0.0668     0.0576783  0.0493439
 0.0480282  0.045924   0.0233717  …  0.0586062  0.0628982  0.0611232
 0.0605082  0.0394706  0.0379736     0.0660183  0.058135   0.0701802
 0.0360314  0.0439399  0.02689       0.0572845  0.0701882  0.0637648
 ⋮                                ⋱                                 
 0.05818

In [89]:
# GRM using all SNPs with MAF ≥ 0.05
grm(hapmap; maf_threshold = 0.05)

324×324 Array{Float64,2}:
 0.569832   0.0534955  0.0229106  …  0.0736094  0.0777382  0.0728604
 0.0534955  0.542797   0.0292553     0.0565556  0.050179   0.0687166
 0.0229106  0.0292553  0.521398      0.0516482  0.03499    0.0406527
 0.0531014  0.0397342  0.0315786     0.061866   0.0725732  0.0652814
 0.0555008  0.0457758  0.0260288     0.0763692  0.063647   0.0720957
 0.048623   0.0330879  0.041643   …  0.0754922  0.0612846  0.0653836
 0.0425229  0.0256048  0.0152978     0.0465058  0.038005   0.0432299
 0.0445646  0.0417373  0.0241854     0.0573711  0.0568992  0.0665918
 0.0315382  0.0336913  0.0200753     0.036809   0.0504817  0.0416076
 0.041739   0.0440421  0.0298305     0.0711053  0.063812   0.0530766
 0.0551959  0.0476215  0.0240236  …  0.0627489  0.0680972  0.0673988
 0.0662055  0.0427204  0.0409013     0.0744466  0.0654627  0.0733776
 0.0363799  0.0475432  0.0301813     0.0635413  0.0731001  0.0677743
 ⋮                                ⋱                                 
 0.06068

In [90]:
# GRM using every other SNP, with maf ≥ 0.01
grm(view(hapmap, :, 1:2:snps))

324×324 Array{Float64,2}:
 0.558921   0.0430417  0.02776    …  0.0675216  0.0737988  0.0675734
 0.0430417  0.557352   0.0283227     0.0581735  0.0459756  0.0563325
 0.02776    0.0283227  0.513037      0.0403644  0.0401923  0.0482823
 0.0453687  0.0459315  0.0274437     0.0522127  0.062348   0.0567562
 0.0525321  0.0482946  0.0261682     0.0675415  0.0565141  0.0658733
 0.0527773  0.0407108  0.0395822  …  0.0763428  0.0629453  0.0538004
 0.0400413  0.0270224  0.0174656     0.0471662  0.0344731  0.0342288
 0.0477958  0.0380087  0.026397      0.0591346  0.0559213  0.0690588
 0.027236   0.0232717  0.0199212     0.0335014  0.0438624  0.0362997
 0.0316181  0.0401905  0.0238639     0.0599384  0.04131    0.0507507
 0.0489757  0.0506289  0.0211802  …  0.0629766  0.0652554  0.0533221
 0.0625952  0.0495802  0.040138      0.0663878  0.0647589  0.0606609
 0.0255679  0.0432178  0.0227905     0.0548997  0.0688556  0.0638942
 ⋮                                ⋱                                 
 0.06032

In [91]:
# MoM using all SNPs with MAF ≥ 0.01
grm(hapmap; method = :MoM)

324×324 Array{Float64,2}:
 0.539557    0.0352687  0.00335917  …  0.0544144  0.0642236  0.0514598
 0.0352687   0.51793    0.0137593      0.0424779  0.0395233  0.0507507
 0.00335917  0.0137593  0.499611       0.0319596  0.0207321  0.0199049
 0.0440143   0.0292413  0.0234504      0.0527598  0.0698964  0.0501598
 0.0455507   0.0333778  0.0156502      0.0656418  0.0571326  0.0567781
 0.0324323   0.0209685  0.0259322   …  0.0593781  0.0495689  0.0488598
 0.0252231   0.0111593  0.00288644     0.0314868  0.0314868  0.0247504
 0.0256958   0.0285322  0.00915016     0.0388142  0.0466143  0.047087 
 0.0209685   0.0253413  0.012932       0.0297141  0.0486234  0.0306595
 0.0210867   0.0255777  0.010332       0.0529962  0.0507507  0.0325505
 0.0364505   0.0337323  0.00548647  …  0.0498053  0.0561872  0.0501598
 0.0488598   0.0323141  0.0275868      0.0559508  0.0553599  0.0524053
 0.0314868   0.0415324  0.0242776      0.0587872  0.0710783  0.0580781
 ⋮                                  ⋱              

## Principal components 

Principal compoenent analysis is widely used in genome-wide association analysis (GWAS) for adjusting population substructure. `pca(A, pcs)` computes the top `pcs` principal components of a `SnpArray`. Each SNP is centered at $2\text{MAF}$ and scaled by $[2\text{MAF}(1-\text{MAF})]^{-1/2}$. The output is  

* `pcscore`: top `pcs` eigen-SNPs, or principal scores, in each column  
* `pcloading`: top `pcs` eigen-vectors, or principal loadings, in each column  
* `pcvariance`: top `pcs` eigen-values, or principal variances

Missing genotypes are imputed according the minor allele frequencies on the fly. This implies that, in the presence of missing genotypes, running the function on the same `SnpArray` twice may produce slightly different answers. For reproducibility, it is a good practice to set the random seed before each function that does imputation on the fly.

In [92]:
srand(123) # set seed
pcscore, pcloading, pcvariance = pca(hapmap, 3)

([-38.7231 -1.2983 -7.00541; -32.6096 -1.21052 -3.3232; … ; -48.9263 -2.06102 2.17374; -48.8627 0.274894 6.49518], [7.47821e-19 1.01333e-18 1.65076e-19; 0.00143962 -0.0042375 -0.00311816; … ; 0.00313326 -0.00427486 -0.0152038; -9.09523e-5 -0.00287777 0.0037855], [1841.4, 225.324, 70.7084])

To use eigen-SNPs for plotting or as covariates in GWAS, we typically scale them by their standard deviations so that they have mean zero and unit variance.

In [93]:
# standardize eigen-SNPs before plotting or GWAS
scale!(pcscore, 1 ./ sqrt.(pcvariance))
std(pcscore, 1)

1×3 Array{Float64,2}:
 1.0  1.0  1.0

Internally `pca` converts `SnpArray` to the matrix of minor allele counts. The default format is `Matrix{Float64}`, which can easily exceed memory limit. Users have several options when the default `Matrix{Float64}` cannot fit into memory.  

* Use other intermediate matrix types.

In [94]:
# use single precision matrix and display the principal variances
# approximately same answer as double precision
srand(123)
pca(hapmap, 3, Matrix{Float32})[3]

3-element Array{Float32,1}:
 1841.4   
  225.324 
   70.7084

* Use subset of SNPs

In [95]:
# principal components using every other SNP capture about half the variance
srand(123)
pca(view(hapmap, :, 1:2:snps), 3)[3]

3-element Array{Float64,1}:
 926.622 
 113.188 
  36.4866

* Use sparse matrix. For large data sets with majority of rare variants, `pca_sp` is more efficient by first converting `SnpArray` to a sparse matrix (default is `SparseMatrixCSC{Float64, Int64}`) and then computing principal components using iterative algorithms. 

In [96]:
# approximately same answer if we use Float16 sparse matrix
srand(123)
pca_sp(hapmap, 3, SparseMatrixCSC{Float32, UInt32})[3]

3-element Array{Float32,1}:
 1841.39  
  225.324 
   70.7084